In [6]:
import warnings 
warnings.filterwarnings('ignore')

import os
import glob
import yfinance as yf
import matplotlib.pyplot as plt 

from tqdm import tqdm
import cv2

In [19]:
try:
    os.mkdir(f'imgs4vid/')
except:
    print('Fold already exists')

In [18]:
data = yf.download("ITUB4.SA",start='2023-10-01').reset_index()

[*********************100%%**********************]  1 of 1 completed


In [20]:
timestep = 30
count=1
for window in tqdm(range(data.shape[0])):
    aux = data[window:window+timestep]
    if aux.shape[0] == timestep:
        stock_prices = aux.copy()
        stock_prices.columns = [col.lower() for col in stock_prices.columns]

        fig = plt.figure()

        up = stock_prices[stock_prices.close >= stock_prices.open]
        down = stock_prices[stock_prices.close < stock_prices.open]
        col1 = 'lightgreen'
        col2 = 'darkred'
        
        width = .6
        width2 = .09

        plt.bar(up.index, up.close-up.open, width, bottom=up.open, color=col1)
        plt.bar(up.index, up.high-up.close, width2, bottom=up.close, color=col1)
        plt.bar(up.index, up.low-up.open, width2, bottom=up.open, color=col1)

        plt.bar(down.index, down.close-down.open, width, bottom=down.open, color=col2)
        plt.bar(down.index, down.high-down.open, width2, bottom=down.open, color=col2)
        plt.bar(down.index, down.low-down.close, width2, bottom=down.close, color=col2)
        
        plt.axis('off')
        plt.savefig(f'imgs4vid/image{count}.jpg')
        # plt.show()
        plt.close()
        count+=1

  0%|          | 0/99 [00:00<?, ?it/s]

100%|██████████| 99/99 [00:23<00:00,  4.15it/s]


In [39]:
img_list = []
for ind in range(len(glob.glob('imgs4vid/*'))):
    img_list.append(f'image{ind+1}.jpg')

In [22]:
def create_video_from_images(image_folder, video_name, fps):
    images = img_list
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

# Specify the folder containing images, name for the output video, and frames per second
image_folder = 'imgs4vid'
video_name = 'itub_candles.mp4'
fps = 2

create_video_from_images(image_folder, video_name, fps)